# Álgebra lineal
## Taller computacional de álgebra lineal y su aplicación en el procesamiento de lenguaje natural (NLP)


**Lea atentamente el siguiente texto introductorio y realice las actividades propuestas en el texto, debes entregar un archivo .ipynb con las actividades resueltas.**

El álgebra lineal es una rama de las matemáticas que estudia los espacios vectoriales y las transformaciones lineales entre ellos. Es fundamental en diversas áreas de la ciencia y la ingeniería, incluyendo la física, la informática, la economía y la estadística.

En la actualidad ha tenido un especial auge debido a su aplicación en el aprendizaje automático y la inteligencia artificial, donde se utilizan conceptos de álgebra lineal para manejar grandes conjuntos de datos y realizar cálculos eficientes. En esta actividad vamos a explorar algunos conceptos básicos del álgebra lineal  el cual seran usados en el desarrollo de modelos que estudien el procesamiento de lenguaje natural (NLP).

Para ello vamos a usar la libreria de Python llamada NumPy, que es una biblioteca fundamental para el cálculo numérico en Python y proporciona soporte para arreglos multidimensionales y funciones matemáticas avanzadas.


## Queremos estudiar 

Para estudiar el proceso de lenguaje natural, vamos a usar unas frases simples y queremos representar cada palabra de las frases en un espacio vectorial. Esto nos permitirá analizar las relaciones entre las palabras y las frases de manera matemática. Para ellos vammos a realizar los siguientes pasos:

1. **Preparación de las frases** a cada frase le vamos a quitar las mayúsculas, tildes y signos de puntuación.

2. **Construcción del vocabulario**: A partir de las frases preparadas, construiremos un vocabulario único de palabras, es decir vamos a listar todas las palabras diferentes que aparecen en las frases y a cada palabra le vamos a signar un índice único. (Esto lo con la función crear_vocabulario).

La siguiente función crea el vocabulario a partir de una lista de frases:

**el rey vive en castillo**
**la reina vive en palacio**

esta son las frases de entrada a la función crear_vocabulario(frases) y el resultado esperado es:
 ['castillo', 'el', 'en', 'la', 'palacio', 'reina', 'rey', 'vive']

 y cada palabra tiene un índice asociado:
 
 {'castillo': 0, 'el': 1, 'en': 2, 'la': 3, 'palacio': 4, 'reina': 5, 'rey': 6, 'vive': 7}



In [35]:
import numpy as np
import matplotlib.pyplot as plt

# ==================== CONFIGURACIÓN INICIAL ====================
frases = [
    "el rey vive en castillo",
    "la reina vive en palacio"
]

# ==================== FUNCIONES DE PREPARACIÓN ====================
def crear_vocabulario(frases):
    """Crear vocabulario a partir de las frases"""
    todas_palabras = []
    for frase in frases:
        palabras = frase.split()
        todas_palabras.extend(palabras)
    
    vocabulario = sorted(list(set(todas_palabras)))
    vocab_size = len(vocabulario)
    palabra_a_idx = {palabra: idx for idx, palabra in enumerate(vocabulario)}
    idx_a_palabra = {idx: palabra for palabra, idx in palabra_a_idx.items()}
    
    print("📚 VOCABULARIO DEL TALLER:")
    print(f"Palabras únicas: {vocabulario}")
    print(f"Total de palabras en vocabulario: {vocab_size}")
    print(f"Dimensión de embeddings: {vocab_size} (sin reducción)")
    
    return vocabulario, vocab_size, palabra_a_idx, idx_a_palabra


vocabulario, vocab_size, palabra_a_idx, idx_a_palabra=crear_vocabulario(frases)


📚 VOCABULARIO DEL TALLER:
Palabras únicas: ['castillo', 'el', 'en', 'la', 'palacio', 'reina', 'rey', 'vive']
Total de palabras en vocabulario: 8
Dimensión de embeddings: 8 (sin reducción)


3. **Representación vectorial de las palabras**: Una vez que tenemos el vocabulario, representaremos cada palabra como un vector one-hot. En esta representación , cada palabra se representa como un vector de longitud igual al tamaño del vocabulario, a cada palabra le corresponde el la frecuencia de sus vecinos en la frase, para esta ocasión usaremos una ventana de tamaño 2 (es decir, consideraremos las dos palabras anteriores y las dos palabras posteriores como vecinos). Por ejemplo, la palabra "vive" en la frase "el rey vive en castillo" tendrá como vecinos "el", "rey", "en" y "castillo".
    La representación vectorial de "vive" sería:
    
    [1, 1, 1, 1, 1, 0, 0, 0]
    
    donde los índices correspondientes a "el", "rey", "en" y "castillo" tienen un valor de 1 (indica que son vecinos) y el resto son 0. 

para realizar esto usaremos la función crear_matriz_co_ocurrencia(frases, vocabulario, palabra_a_idx, ventana=2) que crea una matriz de co-ocurrencia a partir de las frases y el vocabulario generado en el paso anterior.


In [36]:

def crear_matriz_co_ocurrencia(frases, vocabulario, palabra_a_idx, ventana=2):
    """Crear matriz de co-ocurrencia"""
    vocab_size = len(vocabulario)
    co_ocurrencia = np.zeros((vocab_size, vocab_size))
    
    for frase in frases:
        palabras = frase.split()
        for i, palabra in enumerate(palabras):
            if palabra not in palabra_a_idx:
                continue
            idx_actual = palabra_a_idx[palabra]
            
            # Palabras contexto (ventana de 2 palabras alrededor)
            for j in range(max(0, i-ventana), min(len(palabras), i+ventana+1)):
                if i != j and palabras[j] in palabra_a_idx:
                    idx_contexto = palabra_a_idx[palabras[j]]
                    co_ocurrencia[idx_actual][idx_contexto] += 1
    
    return co_ocurrencia
matriz_co_ocurrencia=crear_matriz_co_ocurrencia(frases, vocabulario, palabra_a_idx, ventana=2)
print("Matriz de co-ocurrencia:", matriz_co_ocurrencia)


Matriz de co-ocurrencia: [[0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 1.]
 [1. 0. 0. 0. 1. 1. 1. 2.]
 [0. 0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 1. 1. 0. 0. 0. 1.]
 [0. 1. 1. 0. 0. 0. 0. 1.]
 [1. 1. 2. 1. 1. 1. 1. 0.]]


## Ejercicio 1
Verifica que la tabla de co-ocurrencia generada por la función crear_matriz_co_ocurrencia(frases, vocabulario, palabra_a_idx, ventana=2) es correcta.



De esta forma a cada palabra le vocabulario le podemos asignar su vector de co-ocurrencia correspondiente, por ejemplo:

| Palabra   | Vector de co-ocurrencia               |
|-----------|--------------------------------------|
| castillo  | [0, 0, 1, 0, 0, 0, 0, 1]              |



Note, que este vector asociado a la palabra castillo tambien lo podemos obtener con la función
obtener_embedding_palabra, donde sus parámetros de entrada son la palabra, el idx de la palabra, y la matriz de co-ocurrencia generada anteriormente.

por ejemplo para la palabra "castillo", la variable palabra_a_idx (que contiene el diccionario de mapeo de palabras a índices) y la matriz de co-ocurrencia es la generada anteriormente.

```python
vector_castillo = obtener_embedding_palabra("castillo", palabra_a_idx, matriz_co_ocurrencia)
print("Vector de co-ocurrencia de 'castillo':", vector_castillo)
```


In [37]:
def obtener_embedding_palabra(palabra, palabra_a_idx, embeddings):
    """Obtener embedding de una palabra específica"""
    if palabra in palabra_a_idx:
        idx = palabra_a_idx[palabra]
        return embeddings[idx]
    else:
        return None

vector_castillo = obtener_embedding_palabra("castillo", palabra_a_idx,matriz_co_ocurrencia)
print("Vector de co-ocurrencia de 'castillo':", vector_castillo)

Vector de co-ocurrencia de 'castillo': [0. 0. 1. 0. 0. 0. 0. 1.]


## Ejercicio 2

Supongamos que obtener el vector generado por la frase "la reina vive" usando la matriz de co-ocurrencia generada anteriormente. Para ello, suma los vectores de co-ocurrencia correspondientes a cada palabra en la frase "la reina vive", y se saca el promedio. ¿Cuál es el vector resultante?

##Escribe el codigo en python para realizar este cálculo.

## Ejercicio 3

Genera una forma para ver si las palabras "rey" y "reina" son similares en el espacio vectorial generado por la matriz de co-ocurrencia. ¿Qué método usarías para medir la similitud entre estos dos vectores? Implementa este método y calcula la similitud entre las palabras "rey" y "reina".

### Frases de entrada:
*    "el rey vive en castillo",
*    "la reina vive en palacio", 
*    "hombre trabaja en oficina",
*    "mujer trabaja en empresa",
*    "francia tiene capital paris",
*    "italia tiene capital roma",
*    "coche va por carretera",
*    "barco va por mar",
*    "dinero da riqueza",
*    "trabajo da dinero"

para ello corremos la siguente celda:



In [38]:
## Creación de vocabulario con nuevas frases
frases = [
    "el rey vive en castillo",
    "la reina vive en palacio", 
    "hombre trabaja en oficina",
    "mujer trabaja en empresa",
    "francia tiene capital paris",
    "italia tiene capital roma",
    "coche va por carretera",
    "barco va por mar",
    "dinero da riqueza",
    "trabajo da dinero"
]

vocabulario, vocab_size, palabra_a_idx, idx_a_palabra=crear_vocabulario(frases)
matriz_co_ocurrencia=crear_matriz_co_ocurrencia(frases, vocabulario, palabra_a_idx, ventana=2)

📚 VOCABULARIO DEL TALLER:
Palabras únicas: ['barco', 'capital', 'carretera', 'castillo', 'coche', 'da', 'dinero', 'el', 'empresa', 'en', 'francia', 'hombre', 'italia', 'la', 'mar', 'mujer', 'oficina', 'palacio', 'paris', 'por', 'reina', 'rey', 'riqueza', 'roma', 'tiene', 'trabaja', 'trabajo', 'va', 'vive']
Total de palabras en vocabulario: 29
Dimensión de embeddings: 29 (sin reducción)


## Ejercicio 4

Encuentra el vector que representa los vectores que representan las palabras "rey", "hombre","mujer" y "reina". Ahora, realiza el siguiente cálculo:
**vector_resultante = vector_rey - vector_hombre + vector_mujer**
y comparara el vector_resultante con el vector de la palabra "reina". ¿Son similares? ¿Qué observas?

## Ejercicio 5

+ Realiza el mismo ejercicio anterior pero usando las palabras "castillo", "hombre", "mujer" y "palacio". ¿Qué observas?

+ Realiza el mismo ejercicio anterior pero usando las palabras "paris", "francia"  "italia" Y "roma". ¿Qué observas?
        

## Predicción de palabras

supongamos que tenemos la frase incompleta "el rey vive en ___" y queremos predecir la palabra que falta usando la matriz de co-ocurrencia generada anteriormente. Para ello, sumamos los vectores de co-ocurrencia correspondientes a las palabras presentes en la frase y buscamos la palabra cuyo vector de co-ocurrencia sea más similar al vector resultante.

Para ello, podemos seguir los siguientes pasos:
1. Obtener los vectores de co-ocurrencia para las palabras "el", "rey", "vive" y "en".
2. Sumar estos vectores para obtener un vector resultante.
3. Calcular la similitud entre el vector resultante y los vectores de co-ocurrencia de todas las palabras en el vocabulario.
4. La palabra con la mayor similitud será nuestra predicción para la palabra que falta en la frase.ocurrencia de todas las palabras en el vocabulario. La palabra con la mayor similitud será nuestra predicción para la palabra que falta en la frase.

## Ejercicio 6

Implementa el proceso descrito anteriormente para predecir la palabra que falta en la frase "el rey vive en ___". ¿Cuál es la palabra predicha y qué tan segura estás de esta predicción basado en la similitud calculada?